In [44]:
import cv2
import numpy as np

In [45]:
name='11.jpg'
img = cv2.imread('11.jpg')

In [46]:


def color_balance(img):
    # 1. Calculate the average intensity of each color channel
    red_avg = np.mean(img[:,:,2])
    green_avg = np.mean(img[:,:,1])
    blue_avg = np.mean(img[:,:,0])

    # 2. Compute the difference between the average intensity and the mean intensity
    mean_intensity = np.mean([red_avg, green_avg, blue_avg])
    red_diff = mean_intensity - red_avg
    green_diff = mean_intensity - green_avg
    blue_diff = mean_intensity - blue_avg

    # 3. Adjust the intensity of each channel
    img[:,:,2] = np.clip(img[:,:,2] + red_diff, 0, 255)
    img[:,:,1] = np.clip(img[:,:,1] + green_diff, 0, 255)
    img[:,:,0] = np.clip(img[:,:,0] + blue_diff, 0, 255)

    # 4. Return the processed image
    return img



# Apply color balance
balanced_img = color_balance(img)

variable = name
filename = f'balanced_{variable}.jpg'
cv2.imwrite(filename, balanced_img)

True

In [47]:
import cv2
import numpy as np

def estimate_background_light_for_each_channel(image):
    # Initialize background light for each channel
    background_light_channels = np.zeros(3)
    
    # Convert image to grayscale
    gray_image = image
    
    # Initialize threshold for quadtree division
    threshold = gray_image.size * 0.001
    
    # Define function for quadtree division
    def quadtree_division(image, threshold):
        if image.size <=4:
            return np.mean(image)
        variance = np.var(image)
        
        # If variance is less than threshold, return mean intensity as background light
        if variance < threshold:
            return np.mean(image)
        
        # Otherwise, divide image into four quadrants
        height, width = image.shape
        half_height, half_width = height // 2, width // 2
        quadrants = [
            image[:half_height, :half_width],
            image[:half_height, half_width:],
            image[half_height:, :half_width],
            image[half_height:, half_width:]
        ]
        
        # Recursively call quadtree_division on each quadrant
        background_lights = [quadtree_division(quadrant, threshold) for quadrant in quadrants]
        
        # Return the minimum background light among quadrants
        return min(background_lights)
    
    # Start quadtree division for each channel
    for i in range(3):  # Assuming BGR image
        background_light_channels[i] = quadtree_division(image[:,:,i], threshold)
    
    return background_light_channels

# Example usage:
# Read RGB image
new_name = f'balanced_{name}.jpg'
image = cv2.imread(new_name)

# Estimate background light for each channel
background_light_channels = estimate_background_light_for_each_channel(image)
print("Estimated Background Light for Each Channel:", background_light_channels)


c:\vit\sem6\dip\challenging-60\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\vit\sem6\dip\challenging-60\.venv\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\vit\sem6\dip\challenging-60\.venv\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


RecursionError: maximum recursion depth exceeded

In [37]:
def calculate_contrast_and_complex_performance_index(I, t):
    
    # Calculate the number of channels
    num_channels = I.shape[2]
    
    # Initialize arrays to store contrast values Ci for each channel
    Ci = np.zeros(num_channels)
    
    # Loop through each color channel
    for i in range(num_channels):
       
        # Calculate the contrast value Ci for the current channel
        squared_diff_channel = (I[:,:,i] - np.mean(I[:,:,i])) ** 2
        Ci[i] = np.sum(squared_diff_channel / (t ** 2 * I.size))
    
    # Calculate the complex contrast performance index Ec
    Ec = -np.sum(Ci)
    
    return Ci, Ec



In [38]:
def calculate_information_loss_El(I, A, t):
    # Initialize J(x) array
    J = np.zeros_like(I)
    
    # Calculate J(x) using the restoration model equation for each channel
    for i in range(3):  # Assuming RGB image

        J[:,:,i] = (1 / t) * (I[:,:,i] - A[i]) + A[i]
    
    # Ensure J is within the allowable range (0 to 255)
    J_clipped = np.clip(J, 0, 255)
    
    # Calculate information loss function El
    El = 0
    
    # Loop through each color channel
    for i in range(3):  # Assuming RGB image
        # Calculate information loss for each pixel in the channel
        loss_channel = (np.minimum(0, J_clipped[:,:,i])**2) + np.maximum(0, J_clipped[:,:,i] - 255)
        
        # Sum up the information loss for the channel
        El += np.sum(loss_channel)
    
    return El



In [39]:
! pip install pyswarms

In [40]:
from scipy.optimize import minimize





def combined_objective_function(Ec, El, lambda_L):
    # Calculate the combined objective function
    return Ec + lambda_L * El

from scipy.optimize import minimize

def optimize_transmittance(I, A, lambda_L, initial_guess):
    # Define the objective function for scipy.optimize.minimize
    def objective(t):
        # Reshape t back into a 2D matrix
        t = t.reshape(I.shape[:2])

        # Calculate contrast value Ci and complex contrast performance index Ec
        Ci, Ec = calculate_contrast_and_complex_performance_index(I, t)

        # Calculate information loss El
        El = calculate_information_loss_El(I, A, t)

        # Calculate combined objective function E
        E = combined_objective_function(Ec, El, lambda_L)

        return E

    # Flatten the initial guess into a 1D array
    initial_guess_flat = initial_guess.flatten()

    # Use scipy.optimize.minimize to find the optimal t
    result = minimize(objective, initial_guess_flat, method='BFGS')

    # Reshape the optimal t back into a 2D matrix
    t_optimal = result.x.reshape(I.shape[:2])

    # Calculate the optimal Ec, El, and E
    Ci_optimal, Ec_optimal = calculate_contrast_and_complex_performance_index(I, t_optimal)
    El_optimal = calculate_information_loss_El(I, A, t_optimal)
    E_optimal = combined_objective_function(Ec_optimal, El_optimal, lambda_L)

    # Return the best results
    return t_optimal, Ec_optimal, El_optimal, E_optimal

# Example usage:
# Read RGB image
I = cv2.imread(new_name)

# Background light A obtained from Step 1 (assuming some value)
A = background_light_channels

# Tradeoff factor
lambda_L = 5

# Initial guesses for transmittance
# Initial guess for transmittance
constant = 0.1  # or any small value you prefer
initial_guess = np.around(np.clip(np.random.rand(*I.shape[:2]) + constant, constant,1), decimals=4)

# Optimize transmittance
t_optimal, Ec_optimal, El_optimal, E_optimal = optimize_transmittance(I, A, lambda_L,initial_guess)

# Print the results
print("Optimized Transmittance (t*):", t_optimal)
print("Optimal Ec:", Ec_optimal)
print("Optimal El:", El_optimal)
print("Optimal E:", E_optimal)


MemoryError: Unable to allocate 107. GiB for an array with shape (120000, 120000) and data type float64

In [31]:
import cv2
import numpy as np

def restore_image(I, A, t):
    # Initialize the restored image
    restored_image = np.zeros_like(I)

    # Iterate over each pixel in the image
    for i in range(I.shape[0]):
        for j in range(I.shape[1]):
            # Apply the restoration model formula for each channel
            for k in range(3):  # Iterate over R, G, B channels
                restored_image[i, j, k] = (1 / t[i, j]) * (I[i, j, k] - A[k]) + A[k]

    return restored_image

# Example usage:
# Read RGB image
I = cv2.imread(new_name)

# Assuming background light A is already obtained
A = background_light_channels # Example background light (for each channel)


# Restore the image using the restoration model and the calculated transmittance map

restored_image = restore_image(I, A, t_optimal)
restored_name=f'restored_{name}.jpg'
cv2.imwrite(restored_name, restored_image)
# Display or save the restored image
cv2.imshow('Restored Image', restored_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
print(restored_image)

[[[170 164 104]
  [180 173 103]
  [153 148 104]
  ...
  [158 147  85]
  [216 192  59]
  [170 156  79]]

 [[ 15 255  99]
  [160 155 104]
  [194 186 103]
  ...
  [196 177  68]
  [247 217  46]
  [163 151  82]]

 [[174 168 103]
  [160 155 104]
  [189 181 103]
  ...
  [ 93  40   2]
  [136  74 240]
  [158 147  85]]

 ...

 [[129 202  75]
  [121 168 252]
  [130 208  89]
  ...
  [ 62  95 112]
  [ 74  97 108]
  [ 18  76 107]]

 [[100 120 145]
  [101 123 150]
  [ 92 182  39]
  ...
  [ 64 103 123]
  [255  90 138]
  [ 87 103 110]]

 [[ 91 124 165]
  [ 99 115 134]
  [100 121 148]
  ...
  [ 77 106 121]
  [ 26  94 130]
  [229  86 145]]]


In [ ]:
import numpy as np

def calculate_average_red_channel_value(image):
    # Assuming 'image' is a 3D NumPy array representing the image (height x width x channels)
    red_channel = image[:, :, 0]  # Assuming red channel is the first channel (index 0)
    average_red_value = np.mean(red_channel)
    return average_red_value

average_red = calculate_average_red_channel_value(restored_image)
print("Average Red Channel Value:", average_red)

Average Red Channel Value: 109.21555


In [ ]:

def calculate_histogram_threshold(image):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    print(total_pixels)
    # Calculate histogram threshold (h1) as n*0.225%
    histogram_threshold = total_pixels * 0.00115
    
    return  histogram_threshold

histogram_threshold = calculate_histogram_threshold(restored_image)
print("Histogram Threshold:", histogram_threshold)


120000
Histogram Threshold: 138.0


In [ ]:
# r_threshold = 107
# def calculate_histogram_threshold(image):
#     # Calculate total number of pixels
#     height, width, _ = image.shape
#     total_pixels = height * width
#     print(total_pixels)
#     # Calculate histogram threshold (h1) as n*0.225%
#     histogram_threshold = total_pixels * 0.00115
    
#     return  histogram_threshold

# histogram_threshold = calculate_histogram_threshold(dcb)
# print("Histogram Threshold:", histogram_threshold)


In [ ]:
# import numpy as np

# def histogram_stretching(image, R_ave, R_threshold):
#     # Calculate total number of pixels
#     height, width, _ = image.shape
#     total_pixels = height * width
    
#     # Determine if attenuation of red light is slight or heavy
#     if R_ave >= R_threshold:  # Slight attenuation
#         channels = [0, 1, 2]  # Stretching for all channels (R, G, B)
#     else:  # Heavy attenuation
#         channels = [1, 2]  # Stretching for G, B channels only
    
#     # Calculate satisfactory threshold ht as n * 0.225%
#     ht = total_pixels * 0.00225
    
#     # Calculate minimal and maximal scalar values based on ht
#     imin = 0
#     imax = 255
    
#     # If ht is provided and it's greater than zero, update imin and imax
#     if ht > 0:
#         # Calculate lower and upper thresholds based on ht
#         imin = np.min(image)
#         imax = np.max(image)
#         lower_threshold_count = int(ht)
#         upper_threshold_count = int(total_pixels - ht)
#         sorted_pixels = np.sort(image.flatten())
#         imin = sorted_pixels[lower_threshold_count]
#         imax = sorted_pixels[upper_threshold_count]

#     # Apply histogram stretching to selected channels
#     stretched_image = image.copy()
#     for channel in channels:
#         for i in range(height):
#             for j in range(width):
#                 old_value = stretched_image[i, j, channel]
#                 if old_value < imin:
#                     stretched_image[i, j, channel] = 0
#                 elif old_value > imax:
#                     stretched_image[i, j, channel] = 255
#                 else:
#                     stretched_image[i, j, channel] = 255 * (old_value - imin) / (imax - imin)
    
#     return stretched_image

# # Example usage:
# stretched_image = histogram_stretching(dcb, average_red, r_threshold)




In [ ]:
import numpy as np
r_threshold = 0

def histogram_stretching(image, R_ave, R_threshold):
    # Calculate total number of pixels
    height, width, _ = image.shape
    total_pixels = height * width
    
    # Determine if attenuation of red light is slight or heavy
    if R_ave >= R_threshold:  # Slight attenuation
        channels = [0, 1, 2]  # Stretching for all channels (R, G, B)
    else:  # Heavy attenuation
        channels = [1, 2]  # Stretching for G, B channels only
    
    # Calculate satisfactory threshold ht as n * 0.225%
    ht = total_pixels * 0.00225
    
    # Calculate minimal and maximal scalar values based on ht
    imin = 0
    imax = 255
    
    # If ht is provided and it's greater than zero, update imin and imax
    if ht > 0:
        # Calculate lower and upper thresholds based on ht
        imin = np.min(image)
        imax = np.max(image)
        lower_threshold_count = int(ht)
        upper_threshold_count = int(total_pixels - ht)
        sorted_pixels = np.sort(image.flatten())
        imin = sorted_pixels[lower_threshold_count]
        imax = sorted_pixels[upper_threshold_count]

    # Apply histogram stretching to selected channels
    stretched_image = image.copy()
    for channel in channels:
        for i in range(height):
            for j in range(width):
                old_value = stretched_image[i, j, channel]
                if old_value < imin:
                    stretched_image[i, j, channel] = 0
                elif old_value > imax:
                    stretched_image[i, j, channel] = 255
                else:
                    stretched_image[i, j, channel] = 255 * (old_value - imin) / (imax - imin)
    
    return stretched_image

# Example usage:
stretched_image_model = histogram_stretching(restored_image, average_red, r_threshold)




In [ ]:
streched_name=f'stretched_{name}.jpg'
cv2.imwrite(streched_name, stretched_image_model)

True

In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import color

def calculate_rms_contrast(image):
    """
    Calculate the Root Mean Square (RMS) contrast of an image.
    """
    return np.sqrt(np.mean(np.square(image - np.mean(image))))

def calculate_uciqe(image):
    """
    Calculate the Underwater Color Image Quality Evaluation (UCIQE) of an image.
    """
    c1 = 0.4680
    c2 = 0.2745
    c3 = 0.2576
    img_yuv = color.rgb2yuv(image)
    y = img_yuv[:,:,0]
    u = img_yuv[:,:,1]
    v = img_yuv[:,:,2]
    uciqe = c1 * np.std(y) / np.mean(y) + c2 * np.cov(u, v) / (np.std(u) * np.std(v)) + c3 * np.mean(v)
    return uciqe

def calculate_psnr(image_true, image_test):
    """
    Calculate the Peak Signal-to-Noise Ratio (PSNR) between two images.
    """
    return psnr(image_true, image_test)



In [ ]:
def fuu(image1, image2):
    """
    Main function to calculate image quality metrics.
    """
    # Calculate RMS contrast
    rms_contrast1 = calculate_rms_contrast(image1)
    rms_contrast2 = calculate_rms_contrast(image2)
    print(f"RMS contrast of image 1: {rms_contrast1}")
    print(f"RMS contrast of image 2: {rms_contrast2}")

    # Calculate UCIQE
    uciqe1 = calculate_uciqe(image1)
    uciqe2 = calculate_uciqe(image2)
    print(f"UCIQE of image 1: {uciqe1}")
    print(f"UCIQE of image 2: {uciqe2}")

    # Calculate PSNR
    psnr_value = calculate_psnr(image1, image2)
    print(f"PSNR between image 1 and image 2: {psnr_value}")

   

fuu(img, stretched_image_model)

RMS contrast of image 1: 15.576318265089562
RMS contrast of image 2: 44.177682355259584
UCIQE of image 1: [[0.24534417 0.24535076 0.24419867 ... 0.10883901 0.10980822 0.10996243]
 [0.24535076 0.2453794  0.24422585 ... 0.10888598 0.10985345 0.11000683]
 [0.24419867 0.24422585 0.24330792 ... 0.10905831 0.11003872 0.11017365]
 ...
 [0.10883901 0.10888598 0.10905831 ... 0.09381207 0.09287067 0.09304693]
 [0.10980822 0.10985345 0.11003872 ... 0.09287067 0.09428684 0.09445564]
 [0.10996243 0.11000683 0.11017365 ... 0.09304693 0.09445564 0.09474101]]
UCIQE of image 2: [[0.28078693 0.13873432 0.13761515 ... 0.13363631 0.12608118 0.09595774]
 [0.13873432 0.30969605 0.14814454 ... 0.12780133 0.11453609 0.11636461]
 [0.13761515 0.14814454 0.22522155 ... 0.14646533 0.12093957 0.11049589]
 ...
 [0.13363631 0.12780133 0.14646533 ... 0.43069363 0.14025583 0.11586494]
 [0.12608118 0.11453609 0.12093957 ... 0.14025583 0.5266953  0.12699541]
 [0.09595774 0.11636461 0.11049589 ... 0.11586494 0.12699541 0

ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.